# Spatial coincidence assessed by MPP encodings

## Package setup

In [ ]:
import numpy as np
import scipy
from geo_encodings import MPPEncoder
from geo_encodings import draw_shape

## Setup

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_array
from sklearn.cluster import DBSCAN
import shapely
import shapely.wkt
import plotly
import plotly.subplots
from plotly.graph_objs import Scatter, Figure

## Create an embedding for a region

In [ ]:
x0, y0 = 0, 0
x1, y1 = 100, 100
resolution = 10
scale = 10
encoder = MPPEncoder(region=[x0, y0, x1, y1], resolution=resolution, scale=scale, center=True)
print('%d reference points in encoder' % len(encoder))

## Create geometries

In [ ]:
# Create two LineString objects.
xx = [10, 20, 35, 77, 90]
yy = [10, 25, 42, 67, 80]
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(xx, yy))])
wkt = 'LINESTRING(%s)' % xy
ls1 = shapely.from_wkt(wkt)

xx = [15, 29, 46, 77, 85]
yy = [92, 73, 55, 29, 17]
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(xx, yy))])
wkt = 'LINESTRING(%s)' % xy
ls2 = shapely.from_wkt(wkt)

ls1, ls2

In [ ]:
# For each linestring, generate multipoints in its general vicinity.
# We interpolate points along the length of the linestring, and add some 
# random deviation from each. This gives us points that generally follow the
# linestring but which contain some noise.

from scipy.interpolate import interp1d

def generate_multipoint(geom, n=20):
    points = []
    for d in np.linspace(0, geom.length, n):
        p = shapely.line_interpolate_point(geom, d)
        dx = (np.random.random() - 0.5) * 40
        dy = (np.random.random() - 0.5) * 40
        points.append(shapely.affinity.translate(p, xoff=dx, yoff=dy))
    mp = shapely.MultiPoint(points)
    return mp

mp1 = generate_multipoint(ls1, 20)
mp2 = generate_multipoint(ls2, 25)


In [ ]:
def draw_linestring(geom, color='gray', name='linestring'):
    coords = geom.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name=name, 
        mode='lines', marker={'color': color, 'size': 12}, line={'width': 4}
    )
    return trace

def draw_multipoint(geom, color='gray', name='multipoint', symbol='circle'):
    xx = [z.xy[0][0] for z in geom.geoms]
    yy = [z.xy[1][0] for z in geom.geoms]
    trace = Scatter(
        x=xx, y=yy, name=name, 
        mode='markers', marker={'color': color, 'size': 12},
        marker_symbol=symbol
    )
    return trace

In [ ]:
fig = plotly.subplots.make_subplots(1, 1)

# Draw the linestrings and multipoints.
fig.append_trace(draw_linestring(ls1, color='#1E3A8A', name='LineString 1'), 1, 1)
fig.append_trace(draw_linestring(ls2, color='#D97706', name='LineString 2'), 1, 1)
fig.append_trace(draw_multipoint(mp1, color='#7F1D1D', name='MultiPoint 1', symbol='circle'), 1, 1)
fig.append_trace(draw_multipoint(mp2, color='#0F766E', name='MultiPoint 2', symbol='diamond'), 1, 1)

fig['layout']['width'] = 510
fig['layout']['height'] = 470
fig['layout']['xaxis1']['range'] = [0, 100]
fig['layout']['yaxis1']['range'] = [0, 100]
fig['layout']['xaxis1']['title'] = 'x coordinate'
fig['layout']['yaxis1']['title'] = 'y coordinate'

fig

In [ ]:
# Get a matrix of distances between embedding vectors for the four shapes.
encodings = [
    encoder.encode(ls1),
    encoder.encode(mp1),
    encoder.encode(ls2),
    encoder.encode(mp2)
]
distances = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        dd = encodings[i].sparse() - encodings[j].sparse()
        distances[i, j] = scipy.sparse.linalg.norm(dd)
distances

In [ ]:
# Plot the distance matrix as a heat map.
import plotly.express as px
data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(np.round(distances, 1),
                labels=dict(x="shape", y="shape", color="distance"),
                color_continuous_scale='Inferno',
                x=['LS-1', 'MP-1', 'LS-2', 'MP-2'],
                y=['LS-1', 'MP-1', 'LS-2', 'MP-2'],
                text_auto=True
               )
fig.update_xaxes(side="top")
fig.update_traces(textfont_size=14, textfont_color="darkgray")
fig.show()